In [8]:
import importlib
import sign_tools
import numpy as np

importlib.reload(sign_tools)

from sign_tools import *


### Reading PMR data from input file

Reading the original unrotated file...

In [46]:
N = 8
filename = './Inputs/Triangular_Ladder_Heisenberg/Triangular_Heis_n='+str(N)+'.txt'
Coefficients, BinaryVectors , NumOfParticles = parse_pauli_file(filename)
AllPermsBinary , AllDiagsBinary , PureDiagonals = process_pauli_terms(Coefficients , BinaryVectors , NumOfParticles)
InitialTotalCost , CostsQ , CyclesQ = total_cost_from_binary_operators(AllPermsBinary , AllDiagsBinary)
if len(PureDiagonals) > 0:
    AllPermsBinary.append([0]*NumOfParticles)
    AllDiagsBinary.append(PureDiagonals[0])

print(f'The permutations are {AllPermsBinary}')
print(f'The diagonals are {AllDiagsBinary}')
#InitialTotalCost , CostsQ , CyclesQ = total_cost_from_binary_operators(AllPermsBinary , AllDiagsBinary)
print(f'The initial cost is {InitialTotalCost}')
# CyclesOriginal , N = get_all_cycles_from_file(filename)

The total number of particles are: 8
The permutations are [[1, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0]]
The diagonals are [([(1+0j), (-1+0j)], [[0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0]]), ([(1+0j), (-1+0j)], [[0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0]]), ([(1+0j), (-1+0j)], [[0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0]]), ([(1+0j), (-1+0j)], [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0]]), ([(1+0j), (-1+0j)], [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0]]), ([(1+0j), (-1+0j)], [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0, 0]]), ([(1+0j), (-1+0j)], [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0]]), ([(1+0j), (-1+0j)], [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0,

### Make initial rotations

Before full optimization, we start from an enhanced rotations using our two-body unitary $U_2$ with the following structure:

$U_{\text{odd.}} = 1\otimes U_2 \otimes U_2 \otimes \cdots \otimes U_2$

$U_{\text{even.}} = 1\otimes U_2 \otimes U_2 \otimes \cdots \otimes U_2 \otimes 1$

For even or odd number of spins.




In [194]:
from itertools import combinations

# Search over different rotations
def generate_combinations(totalnum, totalpairs):
    all_pairs = [(i, j) for i in range(1, totalnum) for j in range(i + 1, totalnum)]
    # Select totalpairs pairs
    random_pairs = random.sample(all_pairs, totalpairs)
    return random_pairs

BestCost = InitialTotalCost
numberoftrials = 250
AllPermsBinaryT = AllPermsBinary
AllDiagsBinaryT = AllDiagsBinary
BestU2pairs = []

for i in range(numberoftrials):
    r = random.randint(1, int(N/2)+1)
    allpairs = generate_combinations(N , r)
    for pair in allpairs:
        AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , pair)
    RotatedTotalCost , RotatedCostsQ , RotatedCyclesQ = total_cost_from_binary_operators(AllPermsBinaryT , AllDiagsBinaryT)
    if RotatedTotalCost < BestCost:
        BestCost = RotatedTotalCost
        BestAllPermsBinary = AllPermsBinaryT
        BestAllDiagsBinary = AllDiagsBinaryT
        BestU2pairs.append(allpairs)
        print(f'Found a better cost {BestCost} with addition of pairs {allpairs}')

print(f'The best cost is {BestCost} with the best pairs {BestU2pairs}')



Found a better cost 3552.6542545941643 with addition of pairs [(2, 4), (2, 3)]
Found a better cost 2265.408980106035 with addition of pairs [(1, 3), (1, 2)]
Found a better cost 1518.659709910358 with addition of pairs [(2, 4)]
Found a better cost 1445.5117505050205 with addition of pairs [(1, 2), (2, 4), (1, 3)]
Found a better cost 1299.8433297265233 with addition of pairs [(1, 4), (2, 3)]
Found a better cost 878.4067765668221 with addition of pairs [(2, 4), (1, 4), (2, 3)]
The best cost is 878.4067765668221 with the best pairs [[(2, 4), (2, 3)], [(1, 3), (1, 2)], [(2, 4)], [(1, 2), (2, 4), (1, 3)], [(1, 4), (2, 3)], [(2, 4), (1, 4), (2, 3)]]


In [51]:
AllPermsBinaryT = AllPermsBinary
AllDiagsBinaryT = AllDiagsBinary
# for i in range(int((N-1)/2)):
#     U2spins = (2*i+1 , 2*i+2)
#     print(f'The spins are {U2spins}')
#     AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , U2spins)


#AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (1 , 2))
AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (1 , 2))
AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (5 , 6))
#AllPermsBinaryT , AllDiagsBinaryT = apply_CNOT(AllPermsBinaryT , AllDiagsBinaryT , [(3,4)])
#AllPermsBinaryT , AllDiagsBinaryT = apply_CNOT(AllPermsBinaryT , AllDiagsBinaryT , [(4,5)])
#AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (4 , 5))
#AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (0 , 6))
#AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (0 , 7))

#AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (0 , 7))



# AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (3,4))
# AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (0,1))
# AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (5,6))
# AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (7,8))
# AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (7,8))

# for i in range(int((N-1)/4)):
#     U2spins = (4*i+1 , 4*i+3)
#     print(f'The spins are {U2spins}')
#     AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , U2spins)

#AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (1,2))


#AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (0,1))
#AllPermsBinaryT , AllDiagsBinaryT = apply_U2_rotation(AllPermsBinaryT , AllDiagsBinaryT , (5,6))

#AllPermsBinaryT , AllDiagsBinaryT = apply_single_body(AllPermsBinaryT , AllDiagsBinaryT , [1 ,2] , 'H')

# The new cost after rotation:
RotatedTotalCost , RotatedCostsQ , RotatedCyclesQ = total_cost_from_binary_operators(AllPermsBinaryT , AllDiagsBinaryT)

print(f'The cost after rotation is {RotatedTotalCost}')
print(f'The cost after rotation is {RotatedCostsQ}')

The cost after rotation is 13312.000000000002
The cost after rotation is {3: 0.0, 4: 3072.0000000000005, 5: 10240.000000000002}


In [52]:
TotalCost = RotatedTotalCost
BestCost = RotatedTotalCost
AbsoluteBest = BestCost
AbsoluteBestFound = False
AllTransformations = []
BestTransformations = []

MaxIterations = 200
Iteration = 0
Probabilities = [0.5 , 0.25 , 0.25]
# ================= Simulated annealing 
while TotalCost > TotalCost/500 and Iteration < MaxIterations:
    AllPermsBinaryNew , AllDiagsBinaryNew , Transformation = apply_random_transformation(Probabilities , AllPermsBinary , AllDiagsBinary , NumOfParticles)
    TotalCost , CostsQ , CyclesQ = total_cost_from_binary_operators(AllPermsBinaryNew , AllDiagsBinaryNew)

    DeltaCost = TotalCost - BestCost
    TransitionProb = np.min([np.exp(-1.0*DeltaCost*(Iteration/10 + InitialTotalCost/10.0)/InitialTotalCost) , 1])
    print(f'The new cost is {TotalCost}')
    print(f'The transition probability is {TransitionProb}')
    print(' ')
    p = random.random()
    Iteration += 1
    if p < TransitionProb:
        AllPermsBinary = AllPermsBinaryNew
        AllDiagsBinary = AllDiagsBinaryNew
        BestCost = TotalCost
        AllTransformations.append(Transformation)
        TransformationSinceBest.append(Transformation)
        print('The transformation has been accepted!')
        print(' ')
        print(' ')
    if TotalCost < AbsoluteBest:
        AbsoluteBest = TotalCost
        AbsoluteBestAllPermsBinary = AllPermsBinaryNew
        AbsoluteBestAllDiagsBinary = AllDiagsBinaryNew
        BestTransformations.append(TransformationSinceBest)
        TransformationSinceBest = []
        AbsoluteBestFound = True

if AbsoluteBestFound:
    AllPermsBinary = AbsoluteBestAllPermsBinary
    AllDiagsBinary = AbsoluteBestAllDiagsBinary
# Get the pure diagonal term:
PureDiagonals = []
for i in range(len(AllPermsBinary)):
    if AllPermsBinary[i] == [0]*NumOfParticles:
        PureDiagonals = AllDiagsBinary[i]
        IdentityIndex = i

print(f'The best cost is {BestCost}')
print(f'The best transformations is {AllTransformations}')

AllPermsBinary = AllPermsBinary[:IdentityIndex]+AllPermsBinary[IdentityIndex+1:]
AllDiagsBinary = AllDiagsBinary[:IdentityIndex]+AllDiagsBinary[IdentityIndex+1:]

The new cost is 179200.0
The transition probability is 0.0
 
The new cost is 180224.0
The transition probability is 0.0
 
The new cost is 206848.0
The transition probability is 0.0
 
The new cost is 206848.0
The transition probability is 0.0
 
The new cost is 4608.000000000001
The transition probability is 1.0
 


/var/folders/y7/l21gv11s0fl71s2rjkzl_w1r0000gn/T/ipykernel_14142/2110979978.py:17: RuntimeWarning: overflow encountered in exp
  TransitionProb = np.min([np.exp(-1.0*DeltaCost*(Iteration/10 + InitialTotalCost/10.0)/InitialTotalCost) , 1])


NameError: name 'TransformationSinceBest' is not defined

In [45]:

PureDiagonals = []
for i in range(len(AllPermsBinaryT)):
    if AllPermsBinaryT[i] == [0]*NumOfParticles:
        PureDiagonals = AllDiagsBinaryT[i]
        IdentityIndex = i

AllPermsBinary = AllPermsBinaryT[:IdentityIndex]+AllPermsBinaryT[IdentityIndex+1:]
AllDiagsBinary = AllDiagsBinaryT[:IdentityIndex]+AllDiagsBinaryT[IdentityIndex+1:]
generate_pauli_file_from_pmr_data(filename.removesuffix(".txt")+'_U2.txt', AllPermsBinary , AllDiagsBinary , PureDiagonals)
